![Département de Mathématiques](https://ktzanev.github.io/logolabopp/math-ulille/math-ulille_100.gif)

# TP 4 - Les équations différentielles autonomes de dimension 2 (suite)

Dans ce quatrième TP nous allons étudier numériquement des équations autonomes du premier ordre de dimension 2.<br>
Il s'agit d'exemples vus dans la fiche B de TD.

**Attention :** Pour que le code entre les exercices soit au maximum similaire et permettre de faire des copier/coller, nous utilisons plusieurs fois les mêmes noms pour des fonctions différentes. **Ceci est une très mauvaise pratique qui mène à des résultats imprévisibles si on n'exécute pas les cellules dans l'ordre.**

On commence par charger les bibliothèques « standards » dont on aura besoin.

In [ ]:
# numpy pour les calculs (vectoriels)
import numpy as np
# matplotlib (référencé comme `plt` ici) pour les graphiques
from matplotlib import pyplot as plt
# la bibliothèque pour tracer les trajectoires
from scipy.integrate import odeint
# la bibliothèque qui permet d'utiliser (le décorateur) @interact
from ipywidgets import interact

## Quelques fonctions utiles

### Le champ de vecteurs

De façon similaire à la feuille de TP1, nous allons créer une fonction `champ_vectoriel`, mais avec les différences suivantes :
- la fonction `F` passé en paramètre est la fonction correspondante à l'équation autonome de Cauchy $Y'=F(Y)$.
- elle a un paramètre supplémentaire `normalise` qui indique s'il faut normaliser ou pas le champ de vecteurs.

La fonction `F` passé en paramètre doit opérer sur un **vecteur** `Y` (de dimension $2\times n$) qui représente les $n$ points de la grille où on affiche le champ de vecteurs.

In [ ]:
# affiche le champ de vecteurs sur la figure courante
# - F : représente la fonction de Y'=F(Y)
# - xmin, Xmax, ymin, ymax : détermine la fenêtre d'affichage
# - normalise : si `False` le champ n'est pas normalisé
# - step : est le pas de discrétisations en chaque direction
def champ_vectoriel(F, xmin, xmax, ymin, ymax, normalise=True, step=.5, **kw):
    X = np.arange(xmin, xmax, step)  # abscisses des points de la grille
    Y = np.arange(ymin, ymax, step)  # ordonnées des points de la grille
    U, V = F(np.meshgrid(X, Y))  # les composantes du champ de vecteurs
    if normalise:
        M = np.hypot(U, V)  # calcule la norme du vecteur (U,V)
        M[M == 0] = 1  # évite la division par 0
        U /= M  # normalise la composante U
        V /= M  # normalise la composante V
    return plt.quiver(X, Y, U, V, angles='xy', **kw)

### Les courbes de niveau

La fonction `lignes_niveaux` permet de tracer les courbes de niveaux de la fonction `h` passée en premier paramètre. Nous allons utiliser cette fonction pour dessiner les courbes de niveaux d'un intégral premier ou d'une fonction de Liapounov, par exemple.

In [ ]:
# la fonction qui trace les lignes de niveaux de h qui passe par les points xy
# - h : une fonction de R^2 dans R
# - xmin, xmax, ymin, ymax : le limites du tracé
# - xy : une liste de points de la forme [[x1,x2,...],[y1,y2,...]] par lesquelles passent les niveaux
# - N : le nombre de points de la discrétisation de x et de y
# - fmt : le format de la valeur de la courbe de niveau
# - alpha : le niveau de transparence entre 0 (transparent) et 1 (opaque)
def lignes_niveaux(h, xmin, xmax, ymin, ymax, xy, N=100, fmt='%.1f', alpha=.5):
    C = np.unique(h(np.array(xy)))  # les valeurs de h aux points xy dans l'ordre croissant
    x = np.linspace(xmin, xmax, N)
    y = np.linspace(ymin, ymax, N)
    z = h(np.meshgrid(x, y))  # les valeurs de h aux points (x,y)
    lignes_niveau = plt.contour(x, y, z, levels=C, alpha=alpha)  # les courbes de niveaux
    plt.clabel(lignes_niveau, C, fmt=fmt)  # on ajuste le format des étiquettes des courbes

### Ajustements de F

#### Rajout du paramètre temps

Comme la fonction `odeint` exige deux paramètres, un vecteur et un paramètre temps `t`, on doit rajouter un `t` à la fonction `F` correspondant à l'équation autonome $Y'=F(Y)$. La fonction utilitaire `avec_t` fait ça : on lui passe une fonction à un paramètre `Y` et elle retourne une fonction à deux paramètres : `Y` et `t`.

In [ ]:
avec_t = lambda F: lambda Y, t: F(Y)

### Ralentir le temps en dehors de la fenêtre d'affichage

Si nous voulons afficher les trajectoires dans une fenêtre à priori nous ne savons pas dans quel intervalle il faut faire varier le temps. Si on donne un temps trop court, on n'affiche qu'un bout de l'orbite, si on donne un temps trop long la méthode `odeint` peut « exploser » (soit pour des raisons « théoriques » car il y a une explosion en temps fini, soit pour des raisons « numériques »).

Pour éviter cela nous pouvons modifier `F` en dehors de la fenêtre d'affichage pour éviter ces problèmes, tout en gardant les même orbites dans la fenêtre d'affichage (où `F` reste non modifiée). 

C'est ce qu'on fait avec la fonction `ralentir(F, xmin, xmax, ymin, ymax)` qui retourne une version « ralentie » de `F` en dehors de la fenêtre `[xmin, xmax, ymin, ymax]`.

In [ ]:
# C'est une fonction C^1 qui :
#  - est égale à 1 dans le disque de rayon r
#  - décroit exponentiellement en dehors de ce disque
def cutoff(Y, r):
    nY = np.maximum(Y[0] * Y[0] + Y[1] * Y[1] - r * r, 0)
    return np.exp(-nY)

# la fonction qui « ralentie » F en dehors de la fenêtre [xmin, xmax, ymin, ymax]
def ralentir(F, xmin, xmax, ymin, ymax):
    r = max(abs(xmax), abs(xmin), abs(ymax), abs(ymin)) * np.sqrt(2)  # le rayon d'un disque qui contient la fenêtre
    return lambda Y: F(Y) * cutoff(Y, r)

## Exercice 5 (à faire)

### 1) La fonction de Cauchy

La fonction du problème de Cauchy autonome étudié dans cet exercice est :
$$
    F(Y) = (-y+xy, x+\frac{1}{2}(x^2-y^2)) \text{ pour } Y=(x,y).
$$

Définissez cette fonction.

In [ ]:
# La définition de F

# test
Y = np.array([[0, 1], [0, 1]])  # les arguments (0,0) et (1,1)
R = np.array([[0, 0], [0, 1]])  # le résultat attendu
assert np.all(F(Y) == R), "F(0,0) = (0,0) and F(1,1) = (0,1)"

### 2) La rotation $\pm 120°$

Nous avons vu en TD que ce problème est invariant par la rotation à 120°. Pour pouvoir vérifier cette affirmation nous allons d'abord définir l'application `R120` qui applique la rotation à 120° qui opère sur des vecteurs `Y`.

In [ ]:
# La définition de la rotation R120 qui prend un 2-vecteur comme paramètre

# Ce n'est pas une façon optimale de définir la rotation à -120° = +240°.
# Donc vous pouvez faire mieux, si vous voulez.
R240 = lambda Y: R120(R120(Y))

# test
Y = np.array([[0, 1], [0, 0]])  # R120([1,0]) = [-1/2, sqrt(3)/2]
assert f"{R120(Y)[1,1]:.10f}" == "0.8660254038", f"{R120(Y)[1,1]:.10f} = 0.8660254038 ?"

### 3) Le graphique « dynamique » avec les solutions approchées

Inspirer vous des exemples traités pour afficher un graphique dynamique qui :
- permet de choisir la condition initiale `x_0, y_0` entre -10 et 10 ;
- affiche les isoclines $I_\infty$, $I_{\pm\frac{1}{\sqrt{3}}}$;
- affiche le champ de vecteurs normalisé
- contient la solution approchée obtenue avec `odeint`
- superpose la solution obtenue par rotation de 120° (dont la condition initiale est choisi de façon appropriée).

Le but de ce graphique est de faire la « preuve » visuelle de l'invariance par rotation.

In [ ]:
# choix de la fenêtre
xmin, xmax, ymin, ymax = -10, 10, -10, 10

# choix de la condition initiale grâce à des curseurs
@interact(x_0=(xmin, xmax, .25), y_0=(ymin, ymax, .25))
# le dessin avec la solution pour la condition initiale (x_0,y_0)
def dyn_solution(x_0=1, y_0=np.sqrt(3)):


### 4) Observations

1. Combien de points fixes observez-vous ?
1. Avez-vous pu identifier numériquement les coordonnées de ces points fixes ?
1. Combien d'orbites sont contenus dans l'isocline $I_\infty$ d'après les observations ? (décrivez votre expérience)


**Solution :**


## Exercice 6 (à faire)

La fonction dans cet exercice dépend du paramètre $\varepsilon$ :
$$ 
    F_\varepsilon(Y) = (y, -x+\varepsilon(x^2-1)y) \text{ pour } Y=(x,y).
$$

Vous avez carte blanche pour réaliser l'étude de cet EDO. Inspirez-vous des questions posées dans la feuille de TD.